In [ ]:
import pandas as pd
import re
import os
import time

# Function to process each sheet
def process_sheet(sheet, sheet_name):
    # Find the header row
    header_row = None
    for index, row in sheet.iterrows():
        if ('Entity' in row.values) and ('RTN Level 4' in row.values) and ('Account' in row.values):
            header_row = index
            break

    if header_row is None:
        print(f"Header not found in sheet '{sheet_name}'. Skipping this sheet.")
        return None

    print(f"Found header row at index {header_row} in sheet '{sheet_name}'.")

    # Row above header (year row)
    year_row = header_row - 1

    # Extract headers (Mxx, FY, YTDxx) from col D onwards and concatenate with the year row above
    for col in sheet.columns[3:]:  # Start from column D (index 3)
        # Concatenate the header with the value from the year row
        sheet.at[header_row, col] = f"{sheet.at[header_row, col]} {sheet.at[year_row, col]}"

    # Remove rows above the header row
    sheet = sheet.iloc[header_row:].reset_index(drop=True)

    return sheet

# Main processing function
def main():
    # Get current year and month from user
    current_year = input("Enter current year (e.g., 2024): ")
    current_month = input("Enter current month (1-12): ")

    # Define file paths
    input_excel_file = 'your_file.xlsx'  # Replace with your input file path
    output_file = 'processed_file.xlsx'

    # Check if the output file exists; if so, delete it after closing it
    if os.path.exists(output_file):
        try:
            print(f"Closing and deleting existing file: {output_file}")
            time.sleep(0.5)  # Brief delay to ensure file is not locked
            os.remove(output_file)
            print(f"Deleted existing file: {output_file}")
        except Exception as e:
            print(f"Error deleting the file: {e}")
            return
    else:
        print(f"No existing file found. Creating a new one: {output_file}")

    # Check if the input file exists
    if not os.path.exists(input_excel_file):
        print(f"Input file '{input_excel_file}' not found.")
        return

    # Load the workbook to process
    try:
        xls = pd.ExcelFile(input_excel_file)
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return

    # Process each sheet and write to a new workbook
    try:
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            for sheet_name in xls.sheet_names:
                # Skip sheets with 'excl', 'pivot', or 'tm1' in their names (case-insensitive)
                if re.search(r'(excl|pivot|tm1)', sheet_name, re.IGNORECASE):
                    print(f"Skipping sheet '{sheet_name}' as it contains 'excl', 'pivot', or 'tm1'.")
                    continue

                try:
                    # Read the sheet
                    sheet = pd.read_excel(xls, sheet_name=sheet_name)
                    
                    # Process the sheet
                    processed_sheet = process_sheet(sheet, sheet_name)
                    
                    # If processed successfully, write to the new workbook starting from row 1 (A1)
                    if processed_sheet is not None:
                        processed_sheet.to_excel(writer, sheet_name=sheet_name, index=False)
                except Exception as e:
                    print(f"Error processing sheet '{sheet_name}': {e}")
        print(f"Workbook saved as: {output_file}")
    except Exception as e:
        print(f"Error writing to Excel file: {e}")

if __name__ == "__main__":
    main()
